<a href="https://colab.research.google.com/github/Redwoods/Lec/blob/master/simulation/code/py_modules/py_modules_5_keras_mnist_2_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Keras example: mnist analysisby simple NN (Neural network)

- Keras was designed to easily use the famous deep-learning frameworks; tensorflow, tenano. 
- Keras provides an easy and convenient way to build deep learning models.

    - Keras is an open source python library that enables you to easily build Deep Neural Networks. 
    - The library is capable of running on top of TensorFlow, Theano, Microsoft Cognitive Toolkit, and MXNet. 
    - Tensorflow and Theano are the most used numerical platforms in Python to build Deep Learning algorithms but they can be quite complex and difficult to use.
    
[Goood Intro to Keras](https://towardsdatascience.com/how-to-build-a-neural-network-with-keras-e8faa33d0ae4)

In [0]:
# use TensorFlow 1.x 
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

In [0]:
%%time
from keras.datasets import mnist
(X_train0, y_train0), (X_test0, y_test0) = mnist.load_data()

In [0]:
print(X_train0.shape, X_train0.dtype)
print(y_train0.shape, y_train0.dtype)
print(X_test0.shape, X_test0.dtype)
print(y_test0.shape, y_test0.dtype)

In [0]:
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

In [0]:
plt.figure(figsize=(2, 2))
plt.imshow(X_train0[0], cmap=mpl.cm.bone_r)
plt.grid(False)
plt.xticks([])
plt.yticks([])
plt.show()

In [0]:
y_train0[0]

### Show images of numbers

In [0]:
# 추가 사항
import numpy as np
# import matplotlib as mpl
def plot_digits(instances, images_per_row=10, **options):
    size = 28
    images_per_row = min(len(instances), images_per_row)
    images = [instance.reshape(size,size) for instance in instances]
    n_rows = (len(instances) - 1) // images_per_row + 1
    row_images = []
    n_empty = n_rows * images_per_row - len(instances)
    images.append(np.zeros((size, size * n_empty)))
    for row in range(n_rows):
        rimages = images[row * images_per_row : (row + 1) * images_per_row]
        row_images.append(np.concatenate(rimages, axis=1))
    image = np.concatenate(row_images, axis=0)
    plt.imshow(image, cmap = mpl.cm.binary, **options)
    plt.axis("off")

In [0]:
plt.figure(figsize=(9,9))
example_images = np.r_[X_train0[:50]]
plot_digits(example_images, images_per_row=10)

plt.show()

### 데이터를 float 타입으로 바꾸고 스케일링한다.

In [0]:
X_train = X_train0.reshape(60000, 784).astype('float32') / 255.0
X_test = X_test0.reshape(10000, 784).astype('float32') / 255.0
print(X_train.shape, X_train.dtype)

### y 데이터는 One-Hot-Encoding 을 한다. (Probabilistic labeling)

> One-Hot-Encoding을 하는 이유
- 머신러닝의 목표가 특정 숫자 그림의 숫자를 정확하게 맞추는 것이 아니다.
    1. 일단, 0에서 9까지의 숫자로 판단될 10개의 확률을 계산한다.
    2. 그리고 특정 숫자로 판단될 확률이 제일 큰 수로 결정한다.
- 3개 이상의 클래스를 구분하는 지도학습의 경우 One-Hot-Encoding은 필요한 과정이다.

In [0]:
y_train0[:5]

In [0]:
from keras.utils import np_utils

y_train = np_utils.to_categorical(y_train0, 10)
y_test = np_utils.to_categorical(y_test0, 10)
y_train[:5]

***

## 신경망 구현 순서

### Keras 를 사용하면 다음과 같은 순서로 신경망을 구성할 수 있다.

1. **모형 객체 생성**, Sequential 모형 클래스 객체 생성
2. **신경망 구성**, add 메서드로 layer 추가하여 구성
    - Dense layer 가 가장 일반적인 신경망
    - 입력단부터 순차적으로 추가한다.
    - 레이어는 출력 뉴런 갯수를 첫번째 인수로 받는다.
    - 최초의 레이어는 input_dim 인수로 입력 크기를 설정해야 한다.
    - activation 인수로 activation 함수 설정
3. **compile** 메서드로 모형 완성.
    - loss인수로 Loss 함수 설정
    - optimizer 인수로 최적화 알고리즘 설정
    - metrics 인수로 트레이닝 단계에서 기록할 성능 기준 설정
4. **fit** 메서드로 트레이닝
    - nb_epoch 로 epoch 횟수 설정
    - batch_size 로 mini batch size 설정
    - metrics 인수로 설정한 성능 기준 기록을 출력으로 반환
    - Jupyter Notebook을 사용할 때는 verbose=2로 설정하여 progress bar 가 나오지 않도록 설정한다.

In [0]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
import numpy as np

In [0]:
# Learning model
np.random.seed(0)

model = Sequential()
model.add(Dense(15, input_dim=784, activation="sigmoid"))  # firsr layer
model.add(Dense(10, activation="sigmoid")) # output layer

### 모형 구조 출력 준비
> 만들어진 모형은 model_to_dot 명령이나 summary 명령으로 모델 내부의 layers 리스트를 살펴봄으로써 내부 구조를 확인할 수 있다.


In [0]:
# plot_model(model, to_file='model.png')
# SVG(model_to_dot(model).create(prog='dot', format='svg'))
# - graphviz, pydot 
# Install graphviz, pydot in colab.
# https://laujohn.com/2018/09/24/Plot-Keras-Model-in-Colaboratory/
# Install dependencies
# !apt install graphviz
# !pip install pydot pydot-ng
# !echo "Double check with Python 3"
# !python -c "import pydot"

In [0]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model, show_shapes=True, dpi=70).create(prog='dot', format='svg'))

In [0]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

> 784개의 입력을 받는 하나의 hidden layer(15 neurons)와 출력레이어(10 neurons)로 구성된 단순한 신경망.

In [0]:
model.summary()

In [0]:
l1 = model.layers[0]
l2 = model.layers[1]

In [0]:
l1.name, type(l1), l1.output_shape, l1.activation.__name__, l1.count_params()

In [0]:
# number of parameters
784*15 + 15

In [0]:
l2.name, type(l1), l2.output_shape, l2.activation.__name__, l2.count_params()

In [0]:
# number of parameters
15*10 + 10  

## fit 메서드로 트레이닝

In [0]:
model.compile(optimizer=SGD(lr=0.2), loss='mean_squared_error', metrics=["accuracy"])

In [0]:
%%time
hist = model.fit(X_train, y_train, 
                 epochs=30, batch_size=100, 
                 validation_data=(X_test, y_test), 
                 verbose=1)


# batch_size: 100 -> 60,000개의 트레이닝 데이터를 100개씩 600묶음으로 나눈 후, 
# 각 묶음에 대하어 forward, back propagation을 실시하면서 파라미터를 조정한다.
# epochs: 전체 600개의 batch에 대하여 다 계산을 수행하면 epoch 1 종료.
# validation_data: 테스트 데이터를 섞어서 검증한다.

### Plot of performance and accuracy (모델 성능 및 정확도)

In [0]:
# Plot performance
plt.plot(hist.history['loss'])
plt.show()

In [0]:
plt.plot(hist.history['acc'], 'b-', label="training")
plt.plot(hist.history['val_acc'], 'r:', label="test")
plt.legend()
plt.show()

## 가중치 정보

> 트레이닝이 끝난 모형의 가중치 정보는 get_weights 메서드로 구할 수 있다. 이 메서드는 w 값과 b 값을 출력한다.

In [0]:
w1 = l1.get_weights()  # 784*15 + 15 = 11775
w1[0].shape, w1[1].shape

In [0]:
w2 = l2.get_weights()  # 15*10 + 10 = 160
w2[0].shape, w2[1].shape

## 모형의 사용

> 트레이닝이 끝난 모형은 predict 메서드로 y 값을 출력하거나 출력된 y값을 각 클래스에 대한 판별함수로 가정하고 predict_classes 메서드로 classification을 할 수 있다.

In [0]:
plt.figure(figsize=(2, 2))
plt.imshow(X_test0[0], cmap=mpl.cm.bone_r)
plt.grid(False)
plt.xticks([])
plt.yticks([])
plt.show()

In [0]:
model.predict(X_test[:1, :])  # Probabilitic prediction viq one-hot encoding of labels

In [0]:
model.predict_classes(X_test[:1, :], verbose=0)

### 테스트 데이터에 대한 예측 정확도 계산 

In [0]:
y_pred0 = model.predict(X_test)
y_pred0[:10]

In [0]:
y_pred  = model.predict_classes(X_test, verbose=1)
y_pred[:10]

In [0]:
t_count = np.sum(y_pred == y_test0) # True positive
f_count = np.sum(y_pred != y_test0) # False positive
f_count==10000-t_count

In [0]:
t_count,f_count

In [0]:
accuracy = t_count/10000*100
accuracy

### Accuracy of predicting test numbers is around 89% in NN (neural network).

In [0]:
# see which we predicted correctly and which not
correct_indices = np.nonzero(y_pred == y_test0)[0]
incorrect_indices = np.nonzero(y_pred != y_test0)[0]
print()
print(len(correct_indices)," classified correctly")
print(len(incorrect_indices)," classified incorrectly")

In [0]:
# adapt figure size to accomodate 18 subplots
plt.rcParams['figure.figsize'] = (7,14)

figure_evaluation = plt.figure()

# plot 9 correct predictions
for i, correct in enumerate(correct_indices[:9]):
    plt.subplot(6,3,i+1)
    plt.imshow(X_test[correct].reshape(28,28), cmap='gray', interpolation='none')
    plt.title(
      "Predicted: {}, Truth: {}".format(y_pred[correct],
                                        y_test0[correct]))
    plt.xticks([])
    plt.yticks([])

# plot 9 incorrect predictions
for i, incorrect in enumerate(incorrect_indices[:9]):
    plt.subplot(6,3,i+10)
    plt.imshow(X_test[incorrect].reshape(28,28), cmap='gray', interpolation='none')
    plt.title(
      "Predicted {}, Truth: {}".format(y_pred[incorrect], 
                                       y_test0[incorrect]))
    plt.xticks([])
    plt.yticks([])

# figure_evaluation

## 모형의 저장

>  트레이닝이 끝난 모형은 save 메서드로 가중치와 함께 hdf5 형식으로 저장하였다가 나중에 load 명령으로 불러 사용할 수 있다.

In [0]:
model.save('my_model_NN.hdf5')
# del model

In [0]:
from keras.models import load_model

model2 = load_model('my_model_NN.hdf5')
model2.predict_classes(X_test[:1, :], verbose=0)

In [0]:
model2.predict_classes(X_test[:10, :], verbose=0)

In [0]:
y_test0[:10]

### 저장된 모형을 이용하여 테스트 데이터에 대한 예측 정확도 계산 

In [0]:
# Wrong prediction
model2.predict_classes(X_test[8:9, :], verbose=1)

In [0]:
y_test0[8]  # Compare the true value and the prediction.

In [0]:
# 전체 테스트 데이터에 대한 예측
x_pred = model2.predict_classes(X_test, verbose=1)

In [0]:
t_count = np.sum(x_pred==y_test0) # True positive
f_count = np.sum(x_pred!=y_test0) # False positive
f_count==10000-t_count

In [0]:
t_count,f_count

In [0]:
accuracy = t_count/10000*100
accuracy

### Accuracy of predicting test numbers is around 89% in simple neural network model.



---

